Maclay Teefey (mjt6vj)
Data Project #1 Midterm

# DS-2002 – Data Project 1 100 points


The goal of this project is to demonstrate (1) an understanding of and (2) competence creating and 
implementing basic data science systems such as pipelines, scripts, data transformations, APIs, databases 
and cloud services. Submit your project in your GitHub Repo or file drop on Collab. 
Data Projects must be done individually.

## ETL Data Processor

You project should demonstrate your understanding of the differing types of data systems (OLTP/OLAP), 
and how data can be extracted from various source systems (structured, semi-structured, unstructured), 
transformed (cleansed, integrated), and then loaded into a destination system that’s optimized for post 
hoc diagnostic analysis.

# Deliverable

## 1. Design a dimensional data mart that represents a simple business process of your choosing.

a. Examples might include retail sales, inventory management, procurement, order 
management, transportation or hospitality bookings, medical appointments, student 
registration and/or attendance.

b. You may select any business process that interests you, but remember that a 
dimensional data mart provides for the post hoc summarization and historic analysis of 
business transactions that reflect the interaction between various entities (e.g., patients 
& doctors, retailers & customers, students & schools/classes, travelers & airlines/hotels).

In [96]:
import os
import cryptography
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [97]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "m6zIUutf0k1$"

src_dbname = "world"
dst_dbname = "data_project_warehouse"

In [98]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

The three data sets I will be using and their forms:

1. World sample data set from Oracle in SQL

2. World Happiness Index 2019 data set downloaded as a CSV file from Kaggle (local file system) (link: https://www.kaggle.com/datasets/unsdsn/world-happiness)

3. Nobel Prize Laureates from the Nobel Prizes API (taken in as JSON)

The relationship being warehoused is between the countries and the nobel prize winners. The key joining point will be the country name and birth country of the laureate. The required date portion will be the Birth Date section.

## 2. Develop an ETL pipeline that extracts, transforms, and loads data into your data mart.

a. Extract data from one or more SQL database tables; hosted locally or in the Cloud.

b. Retrieve a data file, either from a remote or local file system, converting its original 
format (e.g., CSV, JSON) into a SQL database table.

c. Modify the number of columns from each source to the destination.

d. Provide error messages wherever an operation fails (i.e., Try/Except error handlers).


Getting the Countries table from World Data

Functions for Getting Data From and Setting Data Into Databases Taken from Lab 03

In [205]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [206]:
sql_world = "SELECT * FROM world.country;"
df_world = get_dataframe(user_id, pwd, host_name, src_dbname, sql_world)
df_world.head(2)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF


Now to get World Happiness data

In [207]:
df_happiness = pd.read_csv("happiness-2019.csv")
df_happiness.head(2)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410


And finally the nobel prize data

In [208]:
# to handle  data retrieval
import urllib3
from urllib3 import request

import numpy as np
# to handle certificate verification
import certifi

# to manage json data
import json

# I am using https://plainenglish.io/blog/from-api-to-pandas-getting-json-data-with-python-df127f699b6b 
# as a guide to convert from JSON to a pandas dataframe

In [209]:
# handle certificate verification and SSL warnings
# https://urllib3.readthedocs.io/en/latest/user-guide.html#ssl

http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

In [210]:
# get data from the API
url = 'https://api.nobelprize.org/v1/laureate.json'

r = http.request('GET', url)
status_check = r.status
status_check # I will be using this as one of the "try catch" without it being a try catch

200

In [211]:
if status_check == 200:
    # decode json data into a dict object
    data = json.loads(r.data.decode('utf-8'))
else:
    print("Error access API")

I would print out the full data but it is too big to print out as a full chunk

In [212]:
# in this dataset, the data to extract is under 'laureates'
df_nobel = pd.json_normalize(data, 'laureates')
df_nobel.head(2)

,id,firstname,surname,born,died,bornCountry,bornCountryCode,bornCity,diedCountry,diedCountryCode,diedCity,gender,prizes
0,1,Wilhelm Conrad,Röntgen,1845-03-27,1923-02-10,Prussia (now Germany),DE,Lennep (now Remscheid),Germany,DE,Munich,male,"[{'year': '1901', 'category': 'physics', 'shar..."
1,2,Hendrik A.,Lorentz,1853-07-18,1928-02-04,the Netherlands,NL,Arnhem,the Netherlands,NL,NaN,male,"[{'year': '1902', 'category': 'physics', 'shar..."


The key that is needed to unite the data sets is in this

In [213]:
df_world["Name"].head(2)

0          Aruba
1    Afghanistan
Name: Name, dtype: object

In [214]:
df_happiness["Country or region"].head(2)

0    Finland
1    Denmark
Name: Country or region, dtype: object

In [215]:
df_nobel["bornCountry"].head(2)

0    Prussia (now Germany)
1          the Netherlands
Name: bornCountry, dtype: object

There is a clear issue with each dataframe having inconsistent naming schemes

I will now get the important details needed from each table and rename the categories into their data mart names.

In [216]:
df_world_trimmed = df_world[["Name", "Code", "Population", "GNP", "LifeExpectancy"]].copy()
df_world_trimmed.reset_index(inplace=True)
df_world_trimmed.rename(columns={"Name":"Country", "Code":"CountryCode", "index":"CountryKey"}, inplace=True)
df_world_trimmed.head(2)

,CountryKey,Country,CountryCode,Population,GNP,LifeExpectancy
0,0,Aruba,ABW,103000,828.0,78.4
1,1,Afghanistan,AFG,22720000,5976.0,45.9


In [217]:
df_happiness_trimmed = df_happiness[["Country or region", "Score"]].copy()
df_happiness_trimmed.reset_index(inplace=True)
df_happiness_trimmed.rename(columns={"Country or region":"Country", "Score":"HappinessScore", "index":"HappinessKey"}, inplace=True)
df_happiness_trimmed.head(2)

,HappinessKey,Country,HappinessScore
0,0,Finland,7.769
1,1,Denmark,7.600


In [218]:
df_nobel.columns

Index(['id', 'firstname', 'surname', 'born', 'died', 'bornCountry',
       'bornCountryCode', 'bornCity', 'diedCountry', 'diedCountryCode',
       'diedCity', 'gender', 'prizes'],
      dtype='object')

In [219]:
df_nobel_trimmed = df_nobel[["firstname","surname","born", "bornCountry", "bornCountryCode", "bornCity", "gender"]].copy()
df_nobel_trimmed = df_nobel_trimmed.dropna()
df_nobel_trimmed.reset_index(inplace=True)
df_nobel_trimmed.rename(columns={"index":"NobelKey", "born":"BirthDate","bornCity":"BirthLocation", "bornCountry":"Country", "bornCountryCode":"CountryCode"}, inplace=True)
df_nobel_trimmed.loc[df_nobel_trimmed["Country"] == "USA", "Country"] = "United States"
df_nobel_trimmed.head(2)

,NobelKey,firstname,surname,BirthDate,Country,CountryCode,BirthLocation,gender
0,0,Wilhelm Conrad,Röntgen,1845-03-27,Prussia (now Germany),DE,Lennep (now Remscheid),male
1,1,Hendrik A.,Lorentz,1853-07-18,the Netherlands,NL,Arnhem,male


I will create smaller fact tables for nobel and country

In [220]:
df_fact_country = pd.merge(df_world_trimmed,df_happiness_trimmed, how="inner", left_on="Country", right_on="Country")
df_fact_country.head()

,CountryKey,Country,CountryCode,Population,GNP,LifeExpectancy,HappinessKey,HappinessScore
0,1,Afghanistan,AFG,22720000,5976.0,45.9,153,3.203
1,4,Albania,ALB,3401200,3205.0,71.6,106,4.719
2,7,United Arab Emirates,ARE,2441000,37966.0,74.1,20,6.825
3,8,Argentina,ARG,37032000,340238.0,75.1,46,6.086
4,9,Armenia,ARM,3520000,1813.0,66.4,115,4.559


And then I will create a date table based upon the birthDate category

In [221]:
min_date = min(df_nobel_trimmed["BirthDate"].astype(str))
max_date = max(df_nobel_trimmed["BirthDate"].astype(str))
print(str(min_date) + "-" + str(max_date))

1817-11-30-1997-07-12


In [222]:
# Here is the code borrowed from https://stackoverflow.com/questions/47150709/how-to-create-a-calendar-table-date-dimension-in-pandas
# to create a 
def create_date_table(start='2000-01-01', end='2050-12-31'):
        df = pd.DataFrame({"Date": pd.date_range(start, end)})
        df["Day"] = df.Date.dt.day_name()
        df["Week"] = df.Date.dt.weekofyear
        df["Quarter"] = df.Date.dt.quarter
        df["Year"] = df.Date.dt.year
        df["Year_half"] = (df.Quarter + 1) // 2
        return df

In [223]:
df_dim_date = create_date_table(min_date, max_date)
df_dim_date.reset_index(inplace=True)
df_dim_date.rename(columns={"index":"DateKey"}, inplace=True)
df_dim_date.head()

C:\Users\student\AppData\Local\Temp\ipykernel_17968\632955866.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["Week"] = df.Date.dt.weekofyear


,DateKey,Date,Day,Week,Quarter,Year,Year_half
0,0,1817-11-30,Sunday,48,4,1817,2
1,1,1817-12-01,Monday,49,4,1817,2
2,2,1817-12-02,Tuesday,49,4,1817,2
3,3,1817-12-03,Wednesday,49,4,1817,2
4,4,1817-12-04,Thursday,49,4,1817,2


In [224]:
df_dim_date_string = df_dim_date.copy().astype('string')
df_fact_nobel = pd.merge(df_nobel_trimmed,df_dim_date_string, how="inner", left_on="BirthDate", right_on="Date")
df_fact_nobel = df_fact_nobel.drop(["Date"],axis=1)
df_fact_nobel.head()

,NobelKey,firstname,surname,BirthDate,Country,CountryCode,BirthLocation,gender,DateKey,Day,Week,Quarter,Year,Year_half
0,0,Wilhelm Conrad,Röntgen,1845-03-27,Prussia (now Germany),DE,Lennep (now Remscheid),male,9979,Thursday,13,1,1845,1
1,1,Hendrik A.,Lorentz,1853-07-18,the Netherlands,NL,Arnhem,male,13014,Monday,29,3,1853,2
2,2,Pieter,Zeeman,1865-05-25,the Netherlands,NL,Zonnemaire,male,17343,Thursday,21,2,1865,1
3,499,John R.,Mott,1865-05-25,United States,US,"Livingston Manor, NY",male,17343,Thursday,21,2,1865,1
4,3,Henri,Becquerel,1852-12-15,France,FR,Paris,male,12799,Wednesday,51,4,1852,2


In [225]:
df_nobel_trimmed["CountryCode"].head(2)

0    DE
1    NL
Name: CountryCode, dtype: object

In [226]:
df_fact_country["CountryCode"].head(2)

0    AFG
1    ALB
Name: CountryCode, dtype: object

There is an issue with the nobel prize code having 2 letter country codes while the world data has 3 letter country codes

Therefore I will be combining the nobel prize data with https://github.com/stefangabos/world_countries/ country code conversion csv called countries.csv in the countries/en/ directory

In [227]:
df_conversion = pd.read_csv("countries.csv")
df_conversion["alpha2"] = df_conversion["alpha2"].str.upper()
df_conversion["alpha3"] = df_conversion["alpha3"].str.upper()
df_conversion.rename(columns={"alpha2":"CountryCode2", "alpha3":"CountryCode3"}, inplace=True)
df_conversion.head(2)

,id,CountryCode2,CountryCode3,name
0,4,AF,AFG,Afghanistan
1,8,AL,ALB,Albania


In [204]:
df_fact_country = pd.merge(df_fact_country,df_conversion, how="inner", left_on="CountryCode", right_on="CountryCode3")
df_fact_country = df_fact_country.drop(["id", "name","CountryCode3", "CountryCode"],axis=1)
df_fact_country.head()

,CountryKey,Country,Population,GNP,LifeExpectancy,HappinessKey,HappinessScore,CountryCode2
0,1,Afghanistan,22720000,5976.0,45.9,153,3.203,AF
1,4,Albania,3401200,3205.0,71.6,106,4.719,AL
2,7,United Arab Emirates,2441000,37966.0,74.1,20,6.825,AE
3,8,Argentina,37032000,340238.0,75.1,46,6.086,AR
4,9,Armenia,3520000,1813.0,66.4,115,4.559,AM


In [237]:
df_fact_combined = pd.merge(df_fact_nobel,df_fact_country, how="inner", left_on="CountryCode", right_on="CountryCode2")
df_fact_combined.reset_index(inplace=True)
df_fact_combined = df_fact_combined.drop(["Quarter","Population", "CountryCode2","Country_x", "gender"], axis=1) 
df_fact_combined.rename(columns={"index":"id","Country_y":"Country"}, inplace=True) #"CountryCode_x":"CountryKey
df_fact_combined.head()

,id,NobelKey,firstname,surname,BirthDate,CountryCode,BirthLocation,DateKey,Day,Week,Year,Year_half,CountryKey,Country,GNP,LifeExpectancy,HappinessKey,HappinessScore
0,0,0,Wilhelm Conrad,Röntgen,1845-03-27,DE,Lennep (now Remscheid),9979,Thursday,13,1845,1,56,Germany,2133367.0,77.4,16,6.985
1,1,12,Ferdinand,Braun,1850-06-06,DE,Fulda,11876,Thursday,23,1850,1,56,Germany,2133367.0,77.4,16,6.985
2,2,17,Max,von Laue,1879-10-09,DE,Pfaffendorf,22593,Thursday,41,1879,2,56,Germany,2133367.0,77.4,16,6.985
3,3,21,Max,Planck,1858-04-23,DE,Kiel,14754,Friday,16,1858,1,56,Germany,2133367.0,77.4,16,6.985
4,4,22,Johannes,Stark,1874-04-15,DE,Schickenhof,20590,Wednesday,16,1874,1,56,Germany,2133367.0,77.4,16,6.985


In [201]:
df_fact_combined.shape

(894, 18)

Now to add all of the data to the data warehouse

In [238]:
db_operation = "insert"

tables = [('fact_combined', df_fact_combined, 'id'), ('world', df_world_trimmed, 'CountryKey'), ('happiness', df_happiness_trimmed, 'HappinessKey'), ('nobel', df_nobel_trimmed, 'NobelKey'), ('dim_date', df_dim_date, 'DateKey')]

for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

## 3. Author one or more SQL queries (SELECT statements) to demonstrate proper functionality.

a. SELECT data from at least 3 tables (two dimensions; plus the fact table).

b. Perform some type of aggregation (e.g., SUM, COUNT, AVERAGE). This, of course, 
necessitates some form of grouping operation (e.g., GROUP BY <customer.last_name>).


In [240]:
sql_select_statement = """
SELECT `fact_combined`.Country as country_name,
       `world`.Population as population,
       COUNT(*) as count_of_laureates,
       COUNT(CASE WHEN `nobel`.gender = "male" then 1 ELSE NULL END) as num_of_men,
       COUNT(CASE WHEN `nobel`.gender = "female" then 1 ELSE NULL END) as num_of_women
FROM `data_project_warehouse`.`fact_combined`
INNER JOIN `data_project_warehouse`.`world`
ON  `fact_combined`.`CountryKey` = `world`.`CountryKey`
INNER JOIN `data_project_warehouse`.`nobel`
ON  `fact_combined`.`NobelKey` = `nobel`.`NobelKey`
GROUP BY `world`.`country`
ORDER BY count_of_laureates DESC;
"""
df_result_one = get_dataframe(user_id, pwd, host_name, src_dbname, sql_select_statement)
df_result_one.head()

,country_name,population,count_of_laureates,num_of_men,num_of_women
0,United States,278357000,279,264,15
1,United Kingdom,59623400,103,101,2
2,Germany,82164700,84,82,2
3,France,59225700,59,54,5
4,Sweden,8861400,30,28,2


The 3 Tables used in this select statment are:

1. fact_combined with the country name
2. world with the country population
3. nobel with the birth continent of the nobel laurette

I also used another table called countries, but it does not show up in the final product because it was only used to convert the 3 letter country intervals in the world sql file to the nobel's 2 letter country intervals